In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sp
import bayesian_online_changepoint_detection as bcod
% matplotlib inline

In [2]:
def generate_data(T):
    # Specify the hazard function.
    # This is a handle to a function that takes one argument - the number of
    # time increments since the last changepoint - and returns a value in
    # the interval [0,1] that is the probability of changepoint.  Generally
    # you might want to have your hazard function take parameters, so using
    # an anonymous function is helpful.  We're going to just use the simple
    # constant-rate hazard function that gives geomtrically-drawn intervals between changepoints.  
    # We'll specify the rate via a mean.

    lambd = 0.001
    def hazard_func(r):
        return lambd

    # This data is Gaussian with unknown mean and variance.  We are going to
    # use the standard conjugate prior of a normal-inverse-gamma.  Note that
    # one cannot use non-informative priors for changepoint detection in
    # this construction.  The NIG yields a closed-form predictive
    # distribution, which makes it easy to use in this context.  There are
    # lots of references out there for doing this kind of inference - for
    # example Chris Bishop's "Pattern Recognition and Machine Learning" in
    # Chapter 2.  Also, Kevin Murphy's lecture notes.

    mu0    = 0
    kappa0 = 1
    alpha0 = 1
    beta0  = 1

    # This will hold the data.  Preallocate for a slight speed improvement.
    X = np.zeros(T)

    # Store the times of changepoints.  It's useful to see them.
    CP = [0]
    
    # Generate the initial parameters of the Gaussian from the prior.
    curr_ivar = np.random.gamma(alpha0)/beta0;
    curr_mean = pow((kappa0*curr_ivar),(-0.5))*np.random.normal() + mu0;
    
    #The initial run length is zero
    curr_run = 0
    
    #Now, loop forward in time and generate data.
    for t in range(0,T):
        # Get the probability of a new changepoint.
        p = hazard_func(curr_run)

        # Randomly generate a changepoint, perhaps.
        if np.random.uniform() < p:

            # Generate new Gaussian parameters from the prior.
            curr_ivar = np.random.gamma(alpha0)*beta0
            curr_mean = pow((kappa0*curr_ivar), (-0.5))*np.random.normal() + mu0

            # The run length drops back to zero.
            curr_run = 0

            # Add this changepoint to the end of the list.
            CP.append(t)
        else:
            # Increment the run length if there was no changepoint.
            curr_run = curr_run + 1

        # Draw data from the current parameters.
        X[t] = pow(curr_ivar, (-0.5)) * np.random.normal() + curr_mean
    return X , CP

In [3]:
#prepare lists for true changepoints and calculated ones
changepoints = []
result_changepoints = []

In [4]:
for i in range(0,100):
    data, CP = generate_data(5000)
    changepoints.append(CP)
    bayes_detector = bcod.Detector(lag = 30)
    result, most_probable_path = bayes_detector.inference(data)
    result_changepoints.append(most_probable_path)

In [23]:
#get changepoint data from result
found_changepoints = []
for result in result_changepoints:
    single_data = []
    #set second index to get rid of initial and final changepoints
    for i in range(2, len(result) -2):
        if result[i] == 0:
            single_data.append(i)
    found_changepoints.append(single_data)

In [32]:
#get rid of initial changepoints
for data in changepoints:
    data.remove(0)

In [51]:
all_changepoints = sum(map(lambda array: len(array), changepoints))
all_found_changepoints = sum(map(lambda array: len(array), found_changepoints))
print("There are {} changepoints ".format(all_changepoints))
print("There are {} found changepoints".format(all_found_changepoints))

There are 539 changepoints 
There are 498 found changepoints


In [40]:
T = 0
F = 0
FA = 0
for i in range(0,100):
    for changepoint in changepoints[i]:
        if changepoint in found_changepoints[i]:
            T = T + 1
        else:
            F = F + 1
    for found_changepoint in found_changepoints[i]:
        if found_changepoint not in changepoints[i]:
            FA = FA + 1

In [42]:
print("There are {} changepoints were correctly detected with zero error".format(T))
print("There are {} changepoints weren't correctly detected with zero error".format(F))
print("There were {}  false detection of changepoints with zero error".format(FA))

There are 231 changepoints were correctly detected with zero error
There are 308 changepoints weren't correctly detected with zero error
There were 267  false detection of changepoints with zero error


In [52]:
T_10 = 0
F_10 = 0
FA_10 = 0
for i in range(0,100):
    for changepoint in changepoints[i]:
        overlap = 0
        for found_changepoint in found_changepoints[i]:
            if abs(changepoint - found_changepoint) < 10:
                overlap = overlap + 1
        if overlap == 0:
            F_10 = F_10 + 1
        else:
            T_10 = T_10 + 1
FA_10 = all_found_changepoints - T_10       

In [56]:
print("There are {} changepoints were correctly detected with 10 observations error".format(T_10))
print("There are {} changepoints weren't correctly detected with 10 observations error".format(F_10))
print("There were {}  false detection of changepoints with 10 observations error".format(FA_10))

There are 457 changepoints were correctly detected with 10 observations error
There are 82 changepoints weren't correctly detected with 10 observations error
There were 41  false detection of changepoints with 10 observations error


In [54]:
T_50 = 0
F_50 = 0
FA_50 = 0
for i in range(0,100):
    for changepoint in changepoints[i]:
        overlap = 0
        for found_changepoint in found_changepoints[i]:
            if abs(changepoint - found_changepoint) < 50:
                overlap = overlap + 1
        if overlap == 0:
            F_50 = F_50 + 1
        else:
            T_50 = T_50 + 1
FA_50 = all_found_changepoints - T_50  

In [57]:
print("There are {} changepoints were correctly detected with 50 observations error".format(T_50))
print("There are {} changepoints weren't correctly detected with 50 observations error".format(F_50))
print("There were {}  false detection of changepoints with 50 observations error".format(FA_50))

There are 497 changepoints were correctly detected with 50 observations error
There are 42 changepoints weren't correctly detected with 50 observations error
There were 1  false detection of changepoints with 50 observations error
